<a href="https://colab.research.google.com/github/kamilo116/AI/blob/master/Basic_CNN_for_cancer_detection_%5BPyTorch%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import numpy as np 
import pandas as pd
import os
import csv
import sys
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
from torch.utils.data.sampler import SubsetRandomSampler

import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as T
import torchvision.transforms as transforms
from torchvision import models
import timeit


In [0]:
from google.colab import drive, files
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip3 install kaggle


In [0]:
# file_list = os.listdir('/content/drive/My Drive/Colab_data/train/')
# file_ids = []
# for f in file_list:
#     file_ids.append(f.split('.')[0])
# print(len(file_ids))

# print(file_list)
# labels = pd.read_csv('/content/drive/My Drive/Colab_data/train_labels.csv')
# print(len(labels))
# new_id_list = []
# for idi, row in labels.iterrows():
#     #print(row['id'])
#     if row['id'] not in file_ids:
#       labels.drop(idi, inplace=True)
      
#       #file_list.remove(row['id'] + '.tif')
#     else:
#       print("id " + str(idi) + " in filelist" )



# print(len(labels))


In [0]:
#uploaded = files.upload()

In [0]:
#print(labels)
#labels.to_csv('selected_train.csv', sep = '\t', index=False)

In [0]:
# with open('/content/drive/My Drive/Colab_data/selected_train.csv') as f:
#   for line in f.readlines():
#     line.replace('\t', ',')
#     print(line)


In [0]:
! cp 'selected_train.csv' '/content/drive/My Drive/Colab_data/selected_train.csv'

cp: cannot stat 'selected_train.csv': No such file or directory


In [0]:
#! cp /content/drive/My\ Drive/Colab_data/kaggle.json /root/.kaggle/kaggle.json
#! chmod 600 /root/.kaggle/kaggle.json

In [0]:
#! kaggle datasets download --path "/content/drive/My\ Drive/Colab_data" histopathologic-cancer-detection


In [0]:
 ! git clone https://github.com/wang-chen/kervolution.git 


Cloning into 'kervolution'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 53 (delta 2), reused 0 (delta 0), pack-reused 47
Unpacking objects: 100% (53/53), done.


so we have 4 files to work with.
and we will load in some librarys, there will probably mix of pytourch, sklearn.

In [0]:
sys.path.append("kervolution/")
from kervolution import Kerv2d

In [0]:


class LinearKernel(torch.nn.Module):
    def __init__(self):
        super(LinearKernel, self).__init__()
    
    def forward(self, x_unf, w, b):
        t = x_unf.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
        if b is not None:
            return t + b
        return t
        
        
class PolynomialKernel(LinearKernel):
    def __init__(self, cp=2.0, dp=3, train_cp=True):
        super(PolynomialKernel, self).__init__()
        self.cp = torch.nn.parameter.Parameter(torch.tensor(cp, requires_grad=train_cp))
        self.dp = dp

    def forward(self, x_unf, w, b):
        return (self.cp + super(PolynomialKernel, self).forward(x_unf, w, b))**self.dp


class GaussianKernel(torch.nn.Module):
    def __init__(self, gamma):
        super(GaussianKernel, self).__init__()
        self.gamma = torch.nn.parameter.Parameter(
                            torch.tensor(gamma, requires_grad=True))
    
    def forward(self, x_unf, w, b):
        l = x_unf.transpose(1, 2)[:, :, :, None] - w.view(1, 1, -1, w.size(0))
        l = torch.sum(l**2, 2)
        t = torch.exp(-self.gamma * l)
        if b:
            return t + b
        return t
        
       
class KernelConv2d(torch.nn.Conv2d):
    def __init__(self, in_channels, out_channels, kernel_size, kernel_fn=PolynomialKernel,
                 stride=1, padding=0, dilation=1, groups=1, bias=None,
                 padding_mode='zeros'):
        '''
        Follows the same API as torch Conv2d except kernel_fn.
        kernel_fn should be an instance of the above kernels.
        '''
        super(KernelConv2d, self).__init__(in_channels, out_channels, 
                                           kernel_size, stride, padding,
                                           dilation, groups, bias, padding_mode)
        self.kernel_fn = kernel_fn()
   
    def compute_shape(self, x):
        h = (x.shape[2] + 2 * self.padding[0] - 1 * (self.kernel_size[0] - 1) - 1) // self.stride[0] + 1
        w = (x.shape[3] + 2 * self.padding[1] - 1 * (self.kernel_size[1] - 1) - 1) // self.stride[1] + 1
        return h, w
    
    def forward(self, x):
        x_unf = torch.nn.functional.unfold(x, self.kernel_size, self.dilation,self.padding, self.stride)
        h, w = self.compute_shape(x)
        return self.kernel_fn(x_unf, self.weight, self.bias).view(x.shape[0], -1, h, w)

In [0]:
class CancerDataset(Dataset):
    def __init__(self, datafolder, datatype='train', transform = transforms.Compose([transforms.ToTensor()]), labels_dict={}):
        
        os.listdir(datafolder)
        self.datafolder = datafolder
        self.datatype = datatype
        self.image_files_list = [s for s in os.listdir(datafolder)]
        self.transform = transform
        self.labels_dict = labels_dict
        if self.datatype == 'train':
            lab = [labels_dict[i.split('.')[0]] for i in self.image_files_list]
            self.labels = lab
            print(lab)
        else:
            self.labels = [0 for _ in range(len(self.image_files_list))]

    def __len__(self):
        return len(self.image_files_list)

    def __getitem__(self, idx):
        #print("idx " + str(idx))
        #print("self.datafolder " +  self.datafolder)
        #print("self.image_files_list[idx] " + self.image_files_list[idx])
        img_name = os.path.join(self.datafolder, self.image_files_list[idx])
        image = Image.open(img_name)
        image = self.transform(image)
        img_name_short = self.image_files_list[idx].split('.')[0]

        if self.datatype == 'train':
            label = self.labels_dict[img_name_short]
        else:
            label = 0
        return image, label


IMAGE_NOT_FOUND_COUNTER = 0

labels = pd.read_csv('/content/drive/My Drive/Colab_data/selected_train.csv', sep='\t') #, header=None, usecols=[0,1])
#print(labels)
file_list = os.listdir('/content/drive/My Drive/Colab_data/train/')
#print(file_list)
#new_labels = pd.DataFrame(columns=('id', 'label'))

#for idi, label in labels.iteritems():
#    for f in file_list:
#        #f = f.split('.')[0]
#        if str(idi) in f:
#            new_labels.append({'id': idi, 'label': label}, ignore_index=True)
     
#print(new_labels)
#pd.DataFrame(my_list, columns = list("abc"))
#labels = pd.Dataframe(new_labels, columns = )

data_transforms = transforms.Compose([
    #transforms.CenterCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
data_transforms_test = transforms.Compose([
    #transforms.CenterCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])



tr, val = train_test_split(labels.label, stratify=labels.label, test_size=0.1)
print("number of training data: ",len(tr))
print("number of testing  data: ",len(val))
# dictionary with labels and ids of train data
img_class_dict = {k:v for k, v in zip(labels.id, labels.label)}

train_sampler = SubsetRandomSampler(list(tr.index))
valid_sampler = SubsetRandomSampler(list(val.index))
batch_size = 16 #256
num_workers = 0
#print(os.listdir('/content/drive/My Drive/Colab_data/train/'))
dataset = CancerDataset(datafolder='/content/drive/My Drive/Colab_data/train/', datatype='train', transform=data_transforms, labels_dict=img_class_dict)
test_set = CancerDataset(datafolder='/content/drive/My Drive/Colab_data/test/', datatype='test', transform=data_transforms_test)
#print(len(dataset))

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=num_workers)

number of training data:  126
number of testing  data:  15
[1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1]


In [0]:
avg_loss_list = []
acc_list = []

def train(model, train_loader ,loss_fn, optimizer, num_epochs = 1):
    total_loss =0

    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        model.train()

        print(train_loader)

        for t, (x, y) in enumerate(train_loader):
            x_var = Variable(x.type(gpu_dtype))
            y_var = Variable(y.type(gpu_dtype).long())

            scores = model(x_var)
            loss = loss_fn(scores, y_var)
            total_loss += loss.data
            
            if (t + 1) % print_every == 0:
                avg_loss = total_loss/print_every
                print('t = %d, avg_loss = %.4f' % (t + 1, avg_loss) )
                avg_loss_list.append(avg_loss)
                total_loss = 0
                

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        acc = check_accuracy(fixed_model_gpu, valid_loader)
        print('acc = %f' %(acc))
            
def check_accuracy(model, loader):
    print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for x, y in loader:
        x_var = Variable(x.type(gpu_dtype))

        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    acc_list.append(acc)
    return acc
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [0]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # read in N, C, H, W
        return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

In [0]:
print_every = 20
gpu_dtype = torch.cuda.FloatTensor

out_1 = 32
out_2 = 64
out_3 = 128
out_4 = 256

k_size_1 = 3
padding_1 = 1


num_epochs = 6

# self.conv1=PolynomialKernelConv(1,10,5,cp=0.5,dp=2)


fixed_model_base = nn.Sequential( 
    #nn.Kerv2d(3, out_1, kernel_size=k_size_1),                             
    #KernelConv2d(3, out_1, kernel_size=k_size_1),
                #nn.Conv2d(3, out_1, padding= padding_1, kernel_size=k_size_1, stride=1), # out_1-k_size_1+1 = 26
                nn.Kerv2d(3, out_1, padding= padding_1, kernel_size=k_size_1, stride=1), # out_1-k_size_1+1 = 26
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_1),
    #KernelConv2d
    #KernelConv2d(out_1, out_1, kernel_size=k_size_1),
                nn.Kerv2d(out_1 , out_1, padding= padding_1, kernel_size=k_size_1, stride=1), #26 - 4 + 1 = 23
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_1),
      
    #KernelConv2d(out_1, out_1, kernel_size=k_size_1),
                nn.Conv2d(out_1 , out_1, padding= padding_1, kernel_size=k_size_1, stride=1), # 23 -3 = 20
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_1),
    
                nn.MaxPool2d(2, stride=2),
    
    #KernelConv2d(out_1, out_2, kernel_size=k_size_1),
                nn.Conv2d(out_1 , out_2, padding= padding_1, kernel_size=k_size_1, stride=1), # 20 -3 = 17
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_2),
   
    #KernelConv2d(out_2, out_2, kernel_size=k_size_1),
                nn.Conv2d(out_2 , out_2, padding= padding_1, kernel_size=k_size_1, stride=1), # 17 -3 = 14
                nn.ReLU(inplace=True), 
                nn.BatchNorm2d(out_2),
    
    #KernelConv2d(out_2, out_2, kernel_size=k_size_1),
                nn.Conv2d(out_2 , out_2, padding= padding_1, kernel_size=k_size_1, stride=1), # 17 -3 = 14
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_2),
    
                nn.MaxPool2d(2, stride=2),
    
    #KernelConv2d(out_2, out_3, kernel_size=k_size_1),
                nn.Conv2d(out_2 , out_3, padding= padding_1, kernel_size=k_size_1, stride=1),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_3),
              
    #KernelConv2d(out_3, out_3, kernel_size=k_size_1),
                nn.Conv2d(out_3 , out_3, padding= padding_1, kernel_size=k_size_1, stride=1), # 17 -3 = 14
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_3),
    #KernelConv2d(out_3, out_3, kernel_size=k_size_1),
                nn.Conv2d(out_3 , out_3, padding= padding_1, kernel_size=k_size_1, stride=1), # 17 -3 = 14
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_3),
    
                nn.MaxPool2d(2, stride=2),
    #KernelConv2d(out_3, out_4, kernel_size=k_size_1),
                nn.Conv2d(out_3 , out_4, padding= padding_1, kernel_size=k_size_1, stride=1), # 17 -3 = 14
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_4),
    #KernelConv2d(out_4, out_4, kernel_size=k_size_1),
                nn.Conv2d(out_4 , out_4, padding= padding_1, kernel_size=k_size_1, stride=1), # 17 -3 = 14
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_4),
    #KernelConv2d(out_4, out_4, kernel_size=k_size_1),
                nn.Conv2d(out_4 , out_4, padding= padding_1, kernel_size=k_size_1, stride=1), # 17 -3 = 14
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_4),
                        #nn.Conv2d(out_11 , out_12, padding= padding_1, kernel_size=k_size_1, stride=1), # 17 -3 = 14
                        #nn.ReLU(inplace=True),
                        #nn.BatchNorm2d(out_12),
    
                nn.MaxPool2d(2, stride=2), #17/2 = 7
                Flatten(),
                
                nn.Linear(9216,512 ), # affine layer
                nn.ReLU(inplace=True),
                nn.Linear(512,10), # affine layer
                nn.ReLU(inplace=True),
                nn.Linear(10,2), # affine layer
            )
fixed_model_gpu = fixed_model_base.type(gpu_dtype)
print(fixed_model_gpu)
loss_fn = nn.modules.loss.CrossEntropyLoss()
optimizer = optim.RMSprop(fixed_model_gpu.parameters(), lr = 1e-3)

train(fixed_model_gpu, train_loader ,loss_fn, optimizer, num_epochs=num_epochs)
check_accuracy(fixed_model_gpu, valid_loader)

RuntimeError: ignored

In [0]:
print(avg_loss_list,acc_list)

In [0]:
import matplotlib.pyplot as plt

plt.plot([print_every*batch_size*(i+1)/len(tr) for i in range((len(avg_loss_list)))],avg_loss_list)
plt.plot([i+1 for i in range((len(acc_list)))],acc_list)
plt.show()

In [0]:
fixed_model_gpu.eval()
preds = []
for batch_i, (data, target) in enumerate(test_loader):
    data, target = data.cuda(), target.cuda()
    output = fixed_model_gpu(data)

    pr = output[:,1].detach().cpu().numpy()
    for i in pr:
        preds.append(i)
        
test_preds = pd.DataFrame({'imgs': test_set.image_files_list, 'preds': preds})

test_preds['imgs'] = test_preds['imgs'].apply(lambda x: x.split('.')[0])

data_to_submit = pd.read_csv('/content/drive/My Drive/Colab_data//sample_submission.csv')
data_to_submit = pd.merge(data_to_submit, test_preds, left_on='id', right_on='imgs')
data_to_submit = data_to_submit[['id', 'preds']]
data_to_submit.columns = ['id', 'label']
data_to_submit.head()

In [0]:
data_to_submit.to_csv('csv_to_submit.csv', index = False)

#citation
* data parsing and code for submittion are taken from: https://www.kaggle.com/artgor/simple-eda-and-model-in-pytorch